In [ ]:
! pip install pydrive

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#папка с даннымы
datasetFolder = ...

In [ ]:
#папка для логов и сохранением моделей
folderToDump = ...
try:
  os.mkdir(folderToDump)
except:
  pass

In [ ]:
os.listdir(datasetFolder)

In [ ]:
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from IPython import display
import numpy as np
from tqdm import tqdm_notebook as tn
import json

In [ ]:
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

!pip install pyheif
import pyheif

In [ ]:
import random
import numpy as np
import torch.backends.cudnn as cudnn

#подготовка к обучению для воспроизводимости
def set_random_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    random.seed(seed)
    np.random.seed(seed)
    
def prepare_cudnn(deterministic=True, benchmark=False):
    if torch.cuda.is_available():
        # CuDNN reproducibility
        # https://pytorch.org/docs/stable/notes/randomness.html#cudnn
        cudnn.deterministic = deterministic

        # https://discuss.pytorch.org/t/how-should-i-disable-using-cudnn-in-my-code/38053/4
        cudnn.benchmark = benchmark
        
def set_deterministic_behaviour(seed=42):
    set_random_seed(seed)
    prepare_cudnn(deterministic=True, benchmark=False)

In [ ]:
#проверка валидности картинки
def is_valid_file(pathToImage):
  try:
    pilImage = Image.open(pathToImage)
    return True
  except:
   pass
  try:
    imRaw = pyheif.read(pathToImage)
    return True
  except:
    return False

In [ ]:
#загрузка картинки
def load_image(pathToImage):
  try:
    pilImage = Image.open(pathToImage)
    pilImage = pilImage.convert('RGB')
    return pilImage
  except:
    imRaw = pyheif.read(pathToImage)
    pilImage = Image.frombytes(mode=imRaw.mode, size=imRaw.size, data=imRaw.data)
    pilImage = pilImage.convert('RGB')
    return pilImage

In [ ]:
#создание датасета с картинками
dataset = ImageFolder(datasetFolder, transform=transforms.Compose([
                                              transforms.Resize((256,256)),
                                               transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225]),
                                               ] ), loader=load_image,
                      )

In [ ]:
#установка разбиения выборки

In [ ]:
train_size = int(len(dataset) * 0.6)
validation_size = int(len(dataset) * 0.2)
test_size = len(dataset) - train_size - validation_size

In [ ]:
set_deterministic_behaviour()

In [ ]:
#создание разбиений датасета
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, (train_size, validation_size, test_size), )

In [ ]:
batch_size = 8
set_deterministic_behaviour()
data_loader_test = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
set_deterministic_behaviour()
data_loader_train = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
set_deterministic_behaviour()
data_loader_validation = torch.utils.data.DataLoader(validation_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
import torchvision.models as models

In [ ]:
#определение самой модели
model = nn.Sequential( *list(models.resnet50(pretrained=True).children())[:-1],\
                      nn.Flatten(1),
                      nn.Linear(in_features=2048, out_features=len(dataset.classes), bias=True)
                     )


In [ ]:
#настройки параметров обучения
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
epoch_num = 15

In [ ]:
loss_function = nn.CrossEntropyLoss()

In [ ]:
#функция для обучения с переданными параметрами
def train(train_dataloader, test_dataloader, net, optimizer, loss_func, epoch_num, w_decay=0):
    set_deterministic_behaviour()
    net.cuda()

    lr = 0.0001
    test_losses = []
    train_losses = []
    test_acces = []
    for epoch in range(epoch_num):
        if epoch in [0, 7, 11]:
        # if epoch in [0, 35, 80]:
            lr = lr / 10
            optimizer = torch.optim.Adam(net.parameters(), weight_decay=w_decay, lr=lr)

        net.train()
        train_loss = []
        progress = tn(train_dataloader)
        for x, y in progress:
            x = x.cuda()
            y = y.cuda()
            optimizer.zero_grad()
            
            out = net(x)        
  
            loss = loss_func(out, y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            
            progress.set_description("loss: %f.3" % np.mean(train_loss) )
            del(x)
            del(y)
            del(out)
            del(loss)

            
        net.eval()
        test_loss = []
        test_acc = []
        for x, y in tn(test_dataloader):
            x = x.cuda()
            y = y.cuda()
            out = net(x)
            loss = loss_func(out, y)
            test_loss.append(loss.item())
            test_acc.extend(list(map(int, list((torch.argmax(out, dim=1) == y).detach().cpu().numpy()))))
            del(x)
            del(y)
            del(out)
            del(loss)
            
        test_losses.append(np.mean(test_loss))
        train_losses.append(np.mean(train_loss))
        test_acces.append(torch.mean(torch.Tensor(test_acc)))
        
        plt.title("epoch:{} train loss: {}, test_loss: {},\n\
                  test_acc {}".format(epoch, train_losses[-1], test_losses[-1], test_acces[-1]))
        display.clear_output(wait=True)
        plt.plot(test_losses,  label="test")
        plt.plot(train_losses, label="train")
        plt.legend()
        plt.show()
        
        plt.plot(test_acces,  label="test")
        plt.legend()
        plt.show()
        
        torch.save(net.state_dict(), folderToDump+"/" + str(epoch))
        logs = dict()
        logs["trainLoss"] = train_loss
        logs["testLoss"] = test_loss
        logs["testAcc"] = test_acc
        
        with open( folderToDump + "/log" + str(epoch) + ".txt", "w" ) as f:
          json.dump(logs, f)

In [ ]:
#запуск обучения
train(data_loader_train, data_loader_validation, model, optimizer, loss_function, epoch_num )

In [ ]:

accs = []
for i in range(12):
  with open(folderToDump + "/log" + str(i) + ".txt") as f:
    accs.append(np.mean(json.load(f)["testAcc"]))

In [ ]:
plt.plot(accs)
plt.scatter(np.argmax(accs), np.max(accs))
print(np.argmax(accs))

In [ ]:
def test(test_dataloader, net, pathToNet):
    set_deterministic_behaviour()
    with open(pathToNet) as f:
      net.load_state_dict(torch.load(pathToNet))
    net.cuda()

 
    net.eval()
    test_loss = []
    test_acc = []
    for x, y in tn(test_dataloader):
        x = x.cuda()
        y = y.cuda()
        out = net(x)
        predictions = torch.argmax(out, dim=1)
        trueValues = list(map(int, list((predictions == y).detach().cpu().numpy())))
        test_acc.extend(trueValues)
        
        for i, value in enumerate(trueValues):
          if value == 0:
            
            print("pred: " + dataset.classes[predictions[i].item()])
            print("gt: " + dataset.classes[y[i].item()])
            plt.figure(figsize=(8,8))
            plt.imshow(x[i].cpu().numpy().transpose(1,2,0))
            plt.show()
            
    test_acc = torch.mean(torch.Tensor(test_acc))
    

    print(test_acc)


In [ ]:
models = ...
test(data_loader_test, model, pathToModel...)